# Objective

Build a model to automatically predict tags for a given a StackExchange question by using the text of the question.
![alt text](https://cdn.sstatic.net/Sites/stackoverflow/company/img/logos/se/se-logo.svg?v=d29f0785ebb7)

__Dataset Specs__: Over 85,000 questions

__License__

All Stack Exchange user contributions are licensed under [CC-BY-SA 3.0](https://creativecommons.org/licenses/by-sa/3.0/) with [attribution required](http://blog.stackoverflow.com/2009/06/attribution-required/).

<br>

***

In [1]:
# optional step (if you are working on colab)
# mount Google drive
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


# Steps to Follow
1. Loading Data
2. Text Cleaning
3. Merge Tags with Questions
4. Dataset Preparation
5. Feature Engineering using TF-IDF
6. Model Building
    1. Naive Bayes
    2. Logistic Regression
    3. Model Building Summary

7. Final Question Tagging Pipeline



In [3]:
# Importing Required Libraries

# for string matching
import re 

# for handling data
import pandas as pd

# for numerical computing
import numpy as np

# for handling html data
from bs4 import BeautifulSoup

# for NLP related tasks
import spacy
nlp = spacy.load('en_core_web_sm', disable=["tagger", "parser", "ner"])

pd.set_option('display.max_colwidth', 200)

# Loading Data

In [4]:
# load questions
questions_df = pd.read_csv('Questions.csv', encoding = 'latin-1')
print('Shape=>', questions_df.shape)
questions_df.head()

Shape=> (85085, 6)


,Id,OwnerUserId,CreationDate,Score,Title,Body
0,6,5.0,2010-07-19T19:14:44Z,272,The Two Cultures: statistics vs. machine learning?,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Mach..."
1,21,59.0,2010-07-19T19:24:36Z,4,Forecasting demographic census,<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\n...
2,22,66.0,2010-07-19T19:25:39Z,208,Bayesian and frequentist reasoning in plain English,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n
3,31,13.0,2010-07-19T19:28:44Z,138,What is the meaning of p values and t values in statistical tests?,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests...."
4,36,8.0,2010-07-19T19:31:47Z,58,Examples for teaching: Correlation does not mean causation,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth ..."


1. Id: Question ID
2. OwnerUserId: User ID
3. CreationDate: Date of posting question
4. Score: Count of Upvotes received by the question
5. Title: Title of the question
6. Body: Text body of the question

In [5]:
# load tags
tags_df = pd.read_csv('Tags.csv')
print('Shape=>', tags_df.shape)
tags_df.head()

Shape=> (244228, 2)


,Id,Tag
0,1,bayesian
1,1,prior
2,1,elicitation
3,2,distributions
4,2,normality


# Text Cleaning

Let's define a function to clean the text data.

In [6]:
def cleaner(text):

  # take off html tags - Beautiful Soup is a library that makes it easy to scrape information from web pages. 
  text = BeautifulSoup(text).get_text()
  
  # fetch alphabetic characters
  text = re.sub("[^a-zA-Z]", " ", text)

  # convert text to lower case
  text = text.lower()
  
  # removing extra spaces
  text=re.sub("[\s]+"," ",text)

  # creating doc object
  doc=nlp(text)

  # remove stopwords and lemmatize the text
  tokens=[token.lemma_ for token in doc if(token.is_stop==False)]

  return " ".join(tokens)

In [7]:
# Pre-processing Questions
questions_df['cleaned_text'] = questions_df['Body'].apply(cleaner)

In [8]:
questions_df['Body'][1]

"<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\nareas are a lot larger than condensed\nurban areas. Is there a need to account for the area size difference?</li>\n<li>if let's say I have census data\ndating back to 4 - 5 census periods,\nhow far can i forecast it into the\nfuture?</li>\n<li>if some of the census zone change\nlightly in boundaries, how can i\naccount for that change?</li>\n<li>What are the methods to validate\ncensus forecasts? for example, if i\nhave data for existing 5 census\nperiods, should I model the first 3\nand test it on the latter two? or is\nthere another way?</li>\n<li>what's the state of practice in\nforecasting census data, and what are\nsome of the state of the art methods?</li>\n</ul>\n"

In [9]:
questions_df['cleaned_text'][1]

'way forecast demographic census validation calibration technique concern census block vary size rural area lot large condense urban area need account area size difference let s census datum date census period far forecast future census zone change lightly boundary account change method validate census forecast example datum exist census period model test way s state practice forecast census datum state art method'

# Merge Tags with Questions

Let's now explore the tags data.

In [10]:
tags_df.head()

,Id,Tag
0,1,bayesian
1,1,prior
2,1,elicitation
3,2,distributions
4,2,normality


Above the first question has 3 tags associated with it.

In [11]:
# count of unique tags
len(tags_df['Tag'].unique())

1315

In [12]:
tags_df['Tag'].value_counts()

r                            13236
regression                   10959
machine-learning              6089
time-series                   5559
probability                   4217
                             ...  
american-community-survey        1
bayesian-anova                   1
network-layout                   1
2d                               1
clumping                         1
Name: Tag, Length: 1315, dtype: int64

In [13]:
# remove "-" from the tags
tags_df['Tag'] = tags_df['Tag'].apply(lambda x:re.sub("-"," ", x))

In [25]:
# group tags Id wise
tags_df = tags_df.groupby('Id').apply(lambda x:x['Tag'].values).reset_index(name ='tags')
tags_df.head()

,Id,tags
0,1,"[bayesian, prior, elicitation]"
1,2,"[distributions, normality]"
2,3,"[software, open source]"
3,4,"[distributions, statistical significance]"
4,6,[machine learning]


Now all the tags are on 1 line with the associated question.  

In [26]:
# merge tags and questions
df = pd.merge(questions_df, tags_df, how = 'inner', on ='Id')

In [27]:
df.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body,cleaned_text,tags
0,6,5.0,2010-07-19T19:14:44Z,272,The Two Cultures: statistics vs. machine learning?,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Mach...",year read blog post brendan o connor entitle statistic vs machine learn fight discuss difference field andrew gelman respond favorably simon blomberg r s fortune package paraphrase provocatively m...,[machine learning]
1,21,59.0,2010-07-19T19:24:36Z,4,Forecasting demographic census,<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\n...,way forecast demographic census validation calibration technique concern census block vary size rural area lot large condense urban area need account area size difference let s census datum date c...,"[forecasting, population, census]"
2,22,66.0,2010-07-19T19:25:39Z,208,Bayesian and frequentist reasoning in plain English,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n,describe plain english characteristic distinguish bayesian frequentist reason,"[bayesian, frequentist]"
3,31,13.0,2010-07-19T19:28:44Z,138,What is the meaning of p values and t values in statistical tests?,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests....",take statistic course try help fellow student notice subject inspire head desk bang interpret result statistical hypothesis test student easily learn perform calculation require give test hang int...,"[hypothesis testing, t test, p value, interpretation, intuition]"
4,36,8.0,2010-07-19T19:31:47Z,58,Examples for teaching: Correlation does not mean causation,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth ...",old say correlation mean causation teach tend use follow standard example illustrate point numb stork birth rate denmark numb priest america alcoholism start th century note strong correlation num...,"[correlation, teaching]"


In [28]:
df = df[['Id', 'Body', 'cleaned_text', 'tags']]
print('Shape=>', df.shape)
df.head()

Shape=> (85085, 4)


,Id,Body,cleaned_text,tags
0,6,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Mach...",year read blog post brendan o connor entitle statistic vs machine learn fight discuss difference field andrew gelman respond favorably simon blomberg r s fortune package paraphrase provocatively m...,[machine learning]
1,21,<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\n...,way forecast demographic census validation calibration technique concern census block vary size rural area lot large condense urban area need account area size difference let s census datum date c...,"[forecasting, population, census]"
2,22,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n,describe plain english characteristic distinguish bayesian frequentist reason,"[bayesian, frequentist]"
3,31,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests....",take statistic course try help fellow student notice subject inspire head desk bang interpret result statistical hypothesis test student easily learn perform calculation require give test hang int...,"[hypothesis testing, t test, p value, interpretation, intuition]"
4,36,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth ...",old say correlation mean causation teach tend use follow standard example illustrate point numb stork birth rate denmark numb priest america alcoholism start th century note strong correlation num...,"[correlation, teaching]"


There are over 85,000 unique questions and over 1300 tags.

# Dataset Preparation

In [56]:
# check frequency of occurence of each tag
freq= {}
for i in df['tags']:
  for j in i:
    if j in freq.keys():
      freq[j] = freq[j] + 1
    else:
      freq[j] = 1

In [57]:
# Printing the first few key value pairs in a dictionary
type(freq)
dict(list(freq.items())[0:10])
print(str(out))

{'r': 13236, 'regression': 10959, 'machine learning': 6089, 'time series': 5559, 'probability': 4217, 'hypothesis testing': 3869, 'self study': 3732, 'distributions': 3501, 'logistic': 3316, 'classification': 2881}


Let's find out the most frequent tags.

In [58]:
# sort the dictionary in descending order
freq = dict(sorted(freq.items(), key = lambda x:x[1], reverse = True))

In [59]:
# freq.items()
type(freq)
dict(list(freq.items())[0:10])

{'r': 13236,
 'regression': 10959,
 'machine learning': 6089,
 'time series': 5559,
 'probability': 4217,
 'hypothesis testing': 3869,
 'self study': 3732,
 'distributions': 3501,
 'logistic': 3316,
 'classification': 2881}

In [60]:
# Top 10 most frequent tags
common_tags = list(freq.keys())[:10]
common_tags

['r',
 'regression',
 'machine learning',
 'time series',
 'probability',
 'hypothesis testing',
 'self study',
 'distributions',
 'logistic',
 'classification']

We will use only those questions/queries that have the above 10 tags associated with it.

In [61]:
x=[]
y=[]

for i in range(len(df['tags'])):
  
  temp=[]
  for j in df['tags'][i]:
    if j in common_tags:
      temp.append(j)

  if(len(temp) > 1):
    x.append(df['cleaned_text'][i])
    y.append(temp)

In [62]:
# number of questions left
len(x)

11106

In [63]:
x[:5]

['recently start work tuberculosis clinic meet periodically discuss numb tb case currently treat numb test administer etc have like start model count guess unusual unfortunately have little train time series exposure model continuous datum stock price large number count influenza deal case month mean median var distribute like image lose mist time image eat grue have find article address model like have greatly appreciate hear suggestion approach r package use implement approach edit mbq s answer force think carefully be ask get hang monthly count lose actual focus question have like know fairly visible decline onward reflect downward trend overall numb case look like numb case monthly reflect stable process maybe seasonality overall stable present look like process change overall numb case decline monthly count wobble randomness seasonality test s real change process identify decline use trend seasonality estimate numb case upcoming month whew thank bear',
 'look well behave distribut

In [64]:
y[:5]

[['r', 'time series'],
 ['regression', 'distributions'],
 ['distributions', 'probability', 'hypothesis testing'],
 ['hypothesis testing', 'self study'],
 ['r', 'regression', 'time series']]

In [65]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Getting Labels 
y = mlb.fit_transform(y)
y.shape

(11106, 10)

In [66]:
y[0,:]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 1])

In [67]:
mlb.classes_

array(['classification', 'distributions', 'hypothesis testing',
       'logistic', 'machine learning', 'probability', 'r', 'regression',
       'self study', 'time series'], dtype=object)

We can now split the dataset into training set and validation set. 

In [68]:
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x, y, test_size = 0.2, random_state = 0, shuffle = True)

In [69]:
print('x_tr:',len(x_tr),'y_tr:',len(y_tr))
print('x_val:',len(x_val),'y_val:',len(y_val))

x_tr: 8884 y_tr: 8884
x_val: 2222 y_val: 2222


# Feature Engineering using TF-IDF

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [72]:
# initialize TFIDF
word_vectorizer = TfidfVectorizer(max_features = 5000)

In [73]:
# Fitting Vectorizer on Train set
word_vectorizer.fit(x_tr)

TfidfVectorizer(max_features=5000)

In [74]:
# create TF-IDF vectors for Train Set
train_word_features = word_vectorizer.transform(x_tr)

In [75]:
train_word_features

<8884x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 394694 stored elements in Compressed Sparse Row format>

In [76]:
# create TF-IDF vectors for Test Set
test_word_features = word_vectorizer.transform(x_val)
test_word_features

<2222x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 96143 stored elements in Compressed Sparse Row format>

# Model Building

## Naive Bayes

In [77]:
# Importing for modeling
from sklearn.multiclass import OneVsRestClassifier
# There are 10 labels therefore there would be 10 models.  OneVsRestClassifier make sthis simple - just one model does the trick!
# https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

In [78]:
# Defining Model
nb_model = OneVsRestClassifier(MultinomialNB())

In [79]:
# Training Model
nb_model.fit(train_word_features, y_tr)

OneVsRestClassifier(estimator=MultinomialNB())

In [80]:
# Make predictions for train set
train_pred_nb = nb_model.predict_proba(train_word_features)

In [81]:
train_pred_nb[:5]

array([[0.0126031 , 0.02891307, 0.27061835, 0.01691539, 0.01902826,
        0.0189542 , 0.83977794, 0.10787773, 0.05661804, 0.93888556],
       [0.01318221, 0.01174116, 0.03175445, 0.11614595, 0.02701485,
        0.01240274, 0.4907905 , 0.76960763, 0.01636875, 0.54218689],
       [0.0489907 , 0.01734154, 0.02706699, 0.24924995, 0.08497525,
        0.02016399, 0.34565106, 0.58293506, 0.0290269 , 0.02143976],
       [0.0171493 , 0.01593526, 0.05291491, 0.0251168 , 0.05569582,
        0.01905612, 0.46108127, 0.29519191, 0.07140457, 0.61706757],
       [0.00441871, 0.00100686, 0.00296404, 0.00521447, 0.03161639,
        0.00107749, 0.95240176, 0.12210743, 0.00207262, 0.99493878]])

The predictions are in terms of probabilities for each of the 10 tags. Hence we need to have a threshold value to convert these probabilities to 0 or 1.

Let's specify a set of candidate threshold values. We will select the threshold value that performs the best for the train set.

In [82]:
# Function for converting probabilities into classes or tags based on a threshold value
def classify(pred_prob, threshold):
  y_pred_seq = []

  for i in pred_prob:
    temp = []
    for j in i:
      if j >= threshold:
        temp.append(1)
      else:
        temp.append(0)
    y_pred_seq.append(temp)

  return y_pred_seq

In [83]:
# Function for finding optimum value of threshold
def optimum_threshold(actual,pred_prob):
  #define candidate threshold values
  thresholds  = np.arange(0, 0.5, 0.01)

  score = []
  for value in thresholds:
    # Getting classes for each threshold
    pred_classes = classify(pred_prob, value) 
    # Getting F1-score for every threshold
    score.append(f1_score(actual, pred_classes, average = "weighted"))

  return thresholds[score.index(max(score))]    

In [84]:
# Finding Optimum value
print("Optimal threshold=>", optimum_threshold(y_tr, train_pred_nb))

Optimal threshold=> 0.26


In [85]:
# Getting classes using optimum threshold
train_pred_nb_class = classify(train_pred_nb, 0.26)

In [86]:
train_pred_nb_class[:5]

[[0, 0, 1, 0, 0, 0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 1]]

In [87]:
mlb.inverse_transform(np.array(train_pred_nb_class[:5]))

[('hypothesis testing', 'r', 'time series'),
 ('r', 'regression', 'time series'),
 ('r', 'regression'),
 ('r', 'regression', 'time series'),
 ('r', 'time series')]

In [88]:
# Evaluating on Training Set
print("F1-score on Train Set:", f1_score(y_tr, train_pred_nb_class, average = "weighted"))

F1-score on Train Set: 0.7269331449940886


In [89]:
# Make Predictions on Validation Set
val_pred_nb = nb_model.predict_proba(test_word_features)

# Getting Classes
val_pred_nb_class = classify(val_pred_nb, 0.26)

# Evaluating on Validation Set
print("F1-score on Validation Set:", f1_score(y_val, val_pred_nb_class, average = "weighted"))

F1-score on Validation Set: 0.6610000974851565


## Logistic Regression

In [90]:
from sklearn.linear_model import LogisticRegression

In [91]:
# Defining Model
lr_model = OneVsRestClassifier(LogisticRegression())

# Training Model
lr_model.fit(train_word_features, y_tr)

OneVsRestClassifier(estimator=LogisticRegression())

In [93]:
# Make Predictions on Train Set
train_pred_lr = lr_model.predict_proba(train_word_features)

In [94]:
train_pred_lr[:5]

array([[0.02641299, 0.0378046 , 0.33069815, 0.02940799, 0.04033737,
        0.02541881, 0.74799613, 0.14494526, 0.06182729, 0.68275999],
       [0.02068351, 0.02382905, 0.06302467, 0.09016197, 0.03624569,
        0.01990367, 0.49387196, 0.8932357 , 0.0416823 , 0.5613228 ],
       [0.09844718, 0.02166288, 0.03934283, 0.21792006, 0.30350662,
        0.02697034, 0.52020303, 0.73508498, 0.06448033, 0.03411671],
       [0.01776753, 0.01264788, 0.35527595, 0.02029543, 0.07045456,
        0.00955896, 0.53486308, 0.37558689, 0.07829327, 0.60630348],
       [0.0269974 , 0.00533884, 0.01979431, 0.02855751, 0.07181888,
        0.00482661, 0.90867834, 0.19564297, 0.01224781, 0.97549888]])

In [95]:
# Finding Optimum value
print("Optimal threshold=>", optimum_threshold(y_tr, train_pred_lr))

Optimal threshold=> 0.33


In [96]:
# Getting classes using optimum threshold
train_pred_lr_class = classify(train_pred_lr, 0.34)
train_pred_lr_class[:5]

[[0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 1, 0, 0, 0, 1, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 1]]

In [97]:
# Evaluating on Training Set
print("F1-score on Train Set:", f1_score(y_tr, train_pred_lr_class, average="weighted"))

F1-score on Train Set: 0.8218502652197971


In [98]:
# Make Predictions on Validation Set
val_pred_lr = lr_model.predict_proba(test_word_features)

# Getting Classes
val_pred_lr_class = classify(val_pred_lr, 0.34)

# Evaluating on Validation Set
print("F1-score on Validation Set:", f1_score(y_val, val_pred_lr_class, average = "weighted"))

F1-score on Validation Set: 0.7562471119584386


## Model Building Summary
|        Model        | Train Set | Validation Set |
|:-------------------:|:---------:|:--------------:|
|     Naive Bayes     |   0.7273  |     0.6622     |
| Logistic Regression |   0.8220  |     0.7562     |

It is evident from the results that Logistic Regression performs better than Naive Bayes.

# Final Question Tagging Pipeline

In [99]:
def tagging(question):
  # Text Cleaning
  cleaned_question = cleaner(question)
  
  # Feature Engineering
  vector = word_vectorizer.transform([cleaned_question])
  
  # Predicting Probabilities
  pred_prob = lr_model.predict_proba(vector)
  
  # Converting Probabilities into classes
  pred_class = classify(pred_prob, 0.34)

  return mlb.inverse_transform(np.array(pred_class))

<font size=4>**Sample Question:**</font>
<p>I'm using SVM classification (Matlab) within my research works, and I want to know:  </p><ol><li>The advantages and disadvantages of each training algorithm, i.e., SMO, LS and QP<br></li><li>In general case, what is the suitable algorithm?<br></li><li>If the choice of one of them can impact the classification performance?<br></li><li>If the are any relationship between the choice of training method (or algorithm) and the choice of the kernel function.</li></ol>

In [100]:
tagging("<p>I'm using SVM classification (Matlab) within my research works, and I want to know:  </p><ol><li>The advantages and disadvantages of each training algorithm, i.e., SMO, LS and QP<br></li><li>In general case, what is the suitable algorithm?<br></li><li>If the choice of one of them can impact the classification performance?<br></li><li>If the are any relationship between the choice of training method (or algorithm) and the choice of the kernel function.</li></ol")

[('classification', 'machine learning')]